This code builds an autonomous agent that is capable of fetching user-level order details.

It uses:


*   GPT model:gpt-3.5-turbo-0125 for parsing and extracting text information from html content
*   Selenium web driver for surfing the web
*   BeautifulSoup for helping the agent with HTML parsing
*   It save order details in Json File







## Imports

In [1]:
!pip install openai
!pip install selenium
!pip install webdriver-manager
!apt install firefox

import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = 'add your key here'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 31.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor firefox libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 8 new

In [5]:
import openai
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from typing import Dict, List

### Web Driver: Chromium

In [3]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,190 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,682 kB]
Hit:14 http://arc

In [4]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

  Using cached chromedriver_autoinstaller-0.6.4-py3-none-any.whl (7.6 kB)


## Coupang Agent

### Parameters

In [24]:
# Set up parameters and prompts for agents
OPENAI_API_KEY = "add your key here"

# Initialize prompts
prompts = {"getorders":"Extract order details using the keyword orderList from this HTML content {}. You should extract key order details from the HTML files, such as order number,product names, quantities, prices, and delivery status, and store them in a structured format (e.g., JSON or CSV). Give me only the file data.",
           "combineorders":"Combine all json files in this text in a single json file and give me only the file without anything else. {}"}


coupang_parameters = {"login_url":"https://login.coupang.com/login/login.pang",
                      "email_inspect":"login-email-input",
                      "pass_inspect":"login-password-input",
                      "orders_url":"https://mc.coupang.com/ssr/desktop/order/list",
                      "nextbuttonclss":'//button[contains(text(), "다음")]',
                      "email":"email.com",
                      "password":"pass",}


# Initialize OpenAI client
client = openai.Client(api_key=OPENAI_API_KEY)


agent_parameters = {"agent_prompts":prompts,"agent":client,"website_parameters":coupang_parameters}

### Utils

In [32]:
#Save json_files in current directory
def save_json(name:str,json_lists:list):
    with open('{}.json'.format(name), 'w') as output_file:
      json.dump(json_lists, output_file)
    return

#Save html files in current directory
def save_html(name:str,html_lists:list):
    for i,html_content in enumerate(html_lists):
      with open('{}{}.html'.format(name,i),'wb+') as output_file:
        output_file.write(html_content)

### Agent Functions

In [33]:
def agent_call(client:openai.Client,prompt:str,format_data:str):
  response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
      {"role": "user", "content": prompt.format(format_data)}
  ],
  temperature=0.2,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )

  return response

def fetch_website(parameters:dict):
  html_files = []

  driver.get(parameters["url"])
  # sign_in_button =  driver.find_element(By.ID,"mycoupang")
  # sign_in_button.click()

  email_input = driver.find_element(By.ID, parameters["email_inspect"])
  email_input.send_keys(parameters["email"])
  email_input.send_keys(Keys.RETURN)

  password_input = driver.find_element(By.ID, parameters["pass_inspect"])
  password_input.send_keys(parameters["password"])
  password_input.send_keys(Keys.RETURN)

  orders_url = parameters["orders_url"] #https://mc.coupang.com/ssr/desktop/order/list
  driver.get(orders_url)

  #get HTML content for first page
  html_files.append(driver.page_source)


  while True:
    try:
      # Wait for the button to be clickable
      button = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, parameters["nextbuttonclss"]))
      )

      # Click the button
      button.click()

      time.sleep(3)  # Wait for the page to load

      #Take the current HTML content
      html_files.append(driver.page_source)
    except:break  # No more pages

  # Close the WebDriver session
  driver.quit()
  return html_files

def fetch_orders(html_content:str,client:openai.Client,agent_prompts):
  order_details = []

  # Find the starting point of JSON data
  start_marker = '</script><script id="__NEXT_DATA__" type="application/json">{'
  start_index = html_content.find(start_marker)

  content_len = len(html_content)
  if start_index != -1:
    # Adjust start_index to start right after the JSON marker
    start_index += len(start_marker)

    # Find the end of JSON data
    end_index = start_index+16385 #16385 is the limit for chatgpt tokens

    while end_index < content_len:
        # Extract the JSON data substring
        json_data_str = html_content[start_index:end_index].strip() #16385 is the limit for chatgpt tokens

        # Parse JSON data
        response = agent_call(client,agent_prompts["getorders"],json_data_str)
        order_details. append(response.choices[0].message.content)

        start_index = end_index
        end_index = start_index+16385

    # Join all seperate json files using the agent
    response = agent_call(client,agent_prompts["combineorders"], order_details)

    try:
      json_data = json.loads(response.choices[0].message.content)
    except:
      json_data = None
    return json_data

def fetch_order_pages(content_list:list,client:openai.Client,agent_prompts:dict) ->list:
  if len(content_list) < 1:
    raise "No order pages available"

  order_details = []
  for html_content in content_list:
    page_response = fetch_orders(html_content,client,agent_prompts)
    order_details.append(page_response)

  return order_details

def automated_agent(agent_parameters:dict):
  #Fetch agent parameters
  client = agent_parameters["agent"]
  agent_prompts = agent_parameters["agent_prompts"]

  #Fetch website html files
  #html_files = fetch_website(agent_parameters["website_parameters"])

  #Save HTML files
  #save_html()

  #Get order details
  order_details = fetch_order_pages([html_content1,html_content2,html_content3],client,agent_prompts)

  #Save as json file
  save_json("order_details",order_details)

  return order_details

In [34]:
ret  = automated_agent(agent_parameters)

In [36]:
ret

[{'orderList': [{'orderId': 4100052853401,
    'title': '아르메데스 남성용 기능성 쿨 민소매나시 AR-121, M, 블랙 + 레드',
    'productList': [{'productName': '아르메데스 남성용 기능성 쿨 민소매 티셔츠 ARPMR-02',
      'quantity': 1,
      'unitPrice': 6500}],
    'deliveryStatus': 'FINAL_DELIVERY'},
   {'orderId': 4100052668763,
    'title': '이더커머스 남녀공용 초경량 풋살 축구화, 275, 블랙 축구화',
    'productList': [{'productName': '이더커머스 남녀공용 초경량 풋살 축구화',
      'quantity': 1,
      'unitPrice': 54900}],
    'deliveryStatus': 'FINAL_DELIVERY'},
   {'orderId': 4100050961705,
    'title': '탐사 샘물, 2L, 30개',
    'productList': [{'productName': '탐사 샘물',
      'quantity': 1,
      'unitPrice': 13490}],
    'deliveryStatus': 'FINAL_DELIVERY'},
   {'orderNumber': '10266589455463',
    'products': [{'productName': '탐사 샘물', 'quantity': 1, 'unitPrice': 13490}],
    'deliveryStatus': 'FINAL_DELIVERY'},
   {'orderNumber': '10266589455312',
    'products': [{'productName': '탐사 샘물', 'quantity': 1, 'unitPrice': 13490}],
    'deliveryStatus': 'FINAL_DELIVERY'

### Test the pipeline

In [22]:
ret = fetch_order_pages([html_content1,html_content2,html_content3])

In [29]:
ret

[{'orderList': [{'orderId': 4100052853401,
    'title': '아르메데스 남성용 기능성 쿨 민소매나시 AR-121, M, 블랙 + 레드',
    'productList': [{'productName': '아르메데스 남성용 기능성 쿨 민소매 티셔츠 ARPMR-02',
      'quantity': 1,
      'unitPrice': 6500}],
    'deliveryGroupList': [{'shipmentBoxId': 6770609764,
      'invoiceNumber': '10267302258603',
      'invoiceStatus': 'FINAL_DELIVERY'}]},
   {'orderId': 4100052668763,
    'title': '이더커머스 남녀공용 초경량 풋살 축구화, 275, 블랙 축구화',
    'productList': [{'productName': '이더커머스 남녀공용 초경량 풋살 축구화',
      'quantity': 1,
      'unitPrice': 54900}],
    'deliveryGroupList': [{'shipmentBoxId': 6764519249,
      'invoiceNumber': '10267233178915',
      'invoiceStatus': 'FINAL_DELIVERY'}]},
   {'orderId': 4100050961705,
    'title': '탐사 샘물, 2L, 30개',
    'productList': [{'productName': '탐사 샘물',
      'quantity': 1,
      'unitPrice': 13490}],
    'deliveryGroupList': [{'shipmentBoxId': 6708998850,
      'invoiceNumber': '10266589455555',
      'invoiceStatus': 'FINAL_DELIVERY'}]},
   {'orderN

In [21]:
# Read the HTML file
with open('coupangorderspage1.html', 'r', encoding='utf-8') as f:
    html_content1 = f.read()

with open('coupangorderspage2.html', 'r', encoding='utf-8') as f:
    html_content2 = f.read()

with open('coupangorderspage3.html', 'r', encoding='utf-8') as f:
    html_content3 = f.read()

#fetch_order_details(html_content)